## Grayscale image conversion to RGB

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from keras.layers import Conv2D, Input, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import tensorflow as tf

In [3]:
#Load the training data
X = []
for filename in os.listdir('/content/drive/My Drive/gray2rgb/dataset2/train/'):
    x = img_to_array(load_img('/content/drive/My Drive/gray2rgb/dataset2/train/'+filename))
    x = np.pad(x, [(0, 0), (7,7), (0,0)], mode='mean')
    X.append(x)
X = np.array(X, dtype=float)

In [4]:
#Split thw data to training and test
frac = int(0.9*len(X))
Xtrain = X[:frac]
Xtrain = 1.0/255*Xtrain #normalizing the pixels to [-1,1] 
Xtest = X[frac:]
Xtest = 1.0/255*Xtest #normalizing the pixels to [-1,1] 

print(Xtrain.shape, Xtest.shape)

# Changing training image orientation,size etc
imgdata = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)


# Generate training data
batch_size = 60
def image_a_b_gen(batch_size):
    #img = 0
    for batch in imgdata.flow(Xtrain, batch_size=batch_size):
        #img+=1
        #print(img)
        lab = rgb2lab(batch)
        X_batch = lab[:,:,:,0]
        Y_batch = lab[:,:,:,1:] / 128 #Lab images have intesity -128 to 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)


#preparing testing images
Xtest = rgb2lab(Xtest)
Xtest = Xtest[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
ytest = rgb2lab(1.0/255*X[frac:])
ytest = ytest[:,:,:,1:]/128

#get size of input image
img_rows = X.shape[1]
img_cols = X.shape[2]


del X
print(img_rows, img_cols)

(1468, 240, 288, 3) (164, 240, 288, 3)
240 288


In [ ]:
#define the keras model
enc_input = Input(shape=(img_rows,img_cols ,1))
model_enc = Conv2D(filters=64, kernel_size=3,  padding='same', activation='relu',strides = 2)(enc_input)
model_enc = Conv2D(64, (3, 3), activation='relu', padding='same', strides=2)(model_enc)
model_enc = BatchNormalization()(model_enc)
model_enc = Conv2D(128, (3, 3), activation='relu', padding='same')(model_enc)
model_enc = Conv2D(128, (3, 3), activation='relu', padding='same', strides=2)(model_enc)
model_enc = BatchNormalization()(model_enc)
model_enc = Conv2D(256, (3, 3), activation='relu', padding='same')(model_enc)
model_enc = Conv2D(256, (3, 3), activation='relu', padding='same', strides=2)(model_enc)
model_enc = BatchNormalization()(model_enc)
model_enc = UpSampling2D(size=(2,2))(model_enc)
model_enc = Conv2D(512, (3, 3), activation='relu', padding='same')(model_enc)
model_enc = Conv2D(512, (3, 3), activation='relu', padding='same')(model_enc)
model_enc = BatchNormalization()(model_enc)
model_enc = UpSampling2D(size=(4,4))(model_enc)
model_enc = Conv2D(256, (3, 3), activation='relu', padding='same')(model_enc)
model_enc = Conv2D(256, (3, 3), activation='relu', padding='same')(model_enc)
model_enc = Conv2D(256, (3, 3), activation='relu', padding='same')(model_enc)
model_enc = BatchNormalization()(model_enc)
model_enc = UpSampling2D(size=(2,2))(model_enc)
outputs = Conv2D(2, (1, 1), activation='tanh', padding='same')(model_enc)

## Training for 30 epochs

In [5]:
#trained for 30 epochs
model = Model(inputs = enc_input, outputs=outputs, name='Cielab')
model.summary()

Model: "Cielab"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 288, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 120, 144, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 72, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 72, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 72, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 36, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 36, 128)       512  

In [ ]:
# Train model
batch_size = 30
model.compile(optimizer='rmsprop' , loss = 'mse', metrics='accuracy')
tensorboard = TensorBoard(log_dir="output/third_run")
filepath = "model3.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
model.fit(image_a_b_gen(batch_size), callbacks=[tensorboard, checkpoint], epochs=30, steps_per_epoch=30)

Epoch 1/30
 2/30 [=>............................] - ETA: 1:18 - loss: 0.5809 - accuracy: 0.5700WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 1.1534s vs `on_train_batch_end` time: 2.4644s). Check your callbacks.


30/30 [==============================] - ETA: 0s - loss: 0.2827 - accuracy: 0.5351
Epoch 00001: loss improved from inf to 0.28270, saving model to model3.h5
30/30 [==============================] - 93s 3s/step - loss: 0.2827 - accuracy: 0.5351
Epoch 2/30
30/30 [==============================] - ETA: 0s - loss: 0.0480 - accuracy: 0.5815
Epoch 00002: loss improved from 0.28270 to 0.04803, saving model to model3.h5
30/30 [==============================] - 92s 3s/step - loss: 0.0480 - accuracy: 0.5815
Epoch 3/30
30/30 [==============================] - ETA: 0s - loss: 0.0395 - accuracy: 0.5896
Epoch 00003: loss improved from 0.04803 to 0.03946, saving model to model3.h5
30/30 [==============================] - 92s 3s/step - loss: 0.0395 - accuracy: 0.5896
Epoch 4/30
30/30 [==============================] - ETA: 0s - loss: 0.0369 - accuracy: 0.6020
Epoch 00004: loss improved from 0.03946 to 0.03692, saving model to model3.h5
30/30 [==============================] - 92s 3s/step - loss: 0.036

In [ ]:
print(model.evaluate(Xtest,ytest,batch_size=batch_size))

4/4 [==============================] - 3s 648ms/step - loss: 0.0158 - accuracy: 0.3747
[0.015776006504893303, 0.37472599744796753]


In [ ]:
# Load black and white images
TestImg = []
cnt = 0
for filename in os.listdir('/content/drive/My Drive/gray2rgb/dataset2/test/'):
  #TestImg.append(img_to_array(load_img('/content/drive/My Drive/gray2rgb/dataset/test/'+filename)))
  cnt+=1
  x = img_to_array(load_img('/content/drive/My Drive/gray2rgb/dataset2/test/'+filename))
  x = np.pad(x, [(0, 0), (7,7), (0,0)], mode='mean')
  TestImg.append(x)
  if cnt==2:
    break

TestImg = np.array(TestImg, dtype=float)
TestImg = rgb2lab(1.0/255*TestImg)[:,:,:,0]
TestImg = TestImg.reshape(TestImg.shape+(1,))

In [ ]:
# Test model
output = model.predict(TestImg)
output = output * 128

In [ ]:
# Output colorizations
for i in range(len(output)):
  cur = np.zeros((img_rows,img_cols, 3))
  cur[:,:,0] = TestImg[i][:,:,0]
  cur[:,:,1:] = output[i]
  imsave("test"+str(i)+".png", lab2rgb(cur))

## Trained for another 30 epochs using the saved model of previous epochs

In [7]:
#training the above model for more epochs
model1 = Model(inputs = enc_input, outputs=outputs, name='Cielab')
model1.summary()

Model: "Cielab"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 288, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 120, 144, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 72, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 72, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 72, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 36, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 36, 128)       512  

In [9]:
# Train model
from tensorflow import keras
opt = keras.optimizers.RMSprop(learning_rate=0.01)
batch_size = 20
model1.compile(optimizer=opt , loss = 'mse', metrics='accuracy')
tensorboard = TensorBoard(log_dir="output/fourth_run")
filepath = "model4.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
model1.load_weights('/content/drive/My Drive/gray2rgb/model3.h5')
model1.fit(image_a_b_gen(batch_size), callbacks=[tensorboard, checkpoint], epochs=30, steps_per_epoch=74)

Epoch 1/30
74/74 [==============================] - ETA: 0s - loss: 0.0216 - accuracy: 0.6751
Epoch 00001: loss improved from inf to 0.02158, saving model to model4.h5
74/74 [==============================] - 61s 822ms/step - loss: 0.0216 - accuracy: 0.6751
Epoch 2/30
74/74 [==============================] - ETA: 0s - loss: 0.0194 - accuracy: 0.6825
Epoch 00002: loss improved from 0.02158 to 0.01936, saving model to model4.h5
74/74 [==============================] - 59s 799ms/step - loss: 0.0194 - accuracy: 0.6825
Epoch 3/30
74/74 [==============================] - ETA: 0s - loss: 0.0189 - accuracy: 0.6734
Epoch 00003: loss improved from 0.01936 to 0.01888, saving model to model4.h5
74/74 [==============================] - 60s 809ms/step - loss: 0.0189 - accuracy: 0.6734
Epoch 4/30
74/74 [==============================] - ETA: 0s - loss: 0.0170 - accuracy: 0.6794
Epoch 00004: loss improved from 0.01888 to 0.01695, saving model to model4.h5
74/74 [==============================] - 61s 8

In [10]:
print(model1.evaluate(Xtest,ytest,batch_size=batch_size))

# Load black and white images
TestImg = []
cnt = 0
for filename in os.listdir('/content/drive/My Drive/gray2rgb/dataset2/test/'):
  #TestImg.append(img_to_array(load_img('/content/drive/My Drive/gray2rgb/dataset/test/'+filename)))
  cnt+=1
  x = img_to_array(load_img('/content/drive/My Drive/gray2rgb/dataset2/test/'+filename))
  x = np.pad(x, [(0, 0), (7,7), (0,0)], mode='mean')
  TestImg.append(x)
  if cnt==2:
    break

TestImg = np.array(TestImg, dtype=float)
TestImg = rgb2lab(1.0/255*TestImg)[:,:,:,0]
TestImg = TestImg.reshape(TestImg.shape+(1,))

output = model1.predict(TestImg)
output = output * 128

# Output colorizations
for i in range(len(output)):
  cur = np.zeros((img_rows,img_cols, 3))
  cur[:,:,0] = TestImg[i][:,:,0]
  cur[:,:,1:] = output[i]
  imsave("test_"+str(i)+".png", lab2rgb(cur))

9/9 [==============================] - 3s 311ms/step - loss: 0.0182 - accuracy: 0.7167
[0.01817537285387516, 0.7167033553123474]
